# Exploring Data

In [6]:
import pandas as pd

df_idk = pd.read_csv('../data/csv_files/Climate_Ready_Boston_Social_Vulnerability.csv')

ppi = pd.read_csv('../data/csv_files/mapc.health_mapc_ppi_g250.csv')

ppi.columns

ppi.head(1)

,objectid,g250m_id,commtypid,commtype,nhwhi_10,nhaa_10,nhapi_10,lat_10,nhoth_10,ppi5,shape
0,1,144054.0,3,Maturing Suburbs,26.88,0.37,3.03,0.37,1.04,2,3C00000005000000080010006A69000020000000011000...
